In [20]:
import requests
import pandas as pd
import json
pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')
#test change

In [21]:
#get=GEO_ID,NAME,EMP,NATION,PAYANN,STATE,YEAR
apikey = 'af4a7c461bcda111515e0ebeea5e4124fcc267d6'
get = 'GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,SEX_LABEL,ETH_GROUP_LABEL,RACE_GROUP_LABEL,VET_GROUP_LABEL,EMPSZFI_LABEL,YEAR,FIRMPDEMP,FIRMPDEMP_F,RCPPDEMP,RCPPDEMP_F,EMP,EMP_F,PAYANN,PAYANN_F,FIRMPDEMP_S,FIRMPDEMP_S_F,RCPPDEMP_S,RCPPDEMP_S_F,EMP_S,EMP_S_F,PAYANN_S,PAYANN_S_F'
url = f"https://api.census.gov/data/2018/abscs?get={get}&for=us:*&key={apikey}"
r = requests.get(url)
print(r)
r = r.json()

<Response [200]>


In [22]:
#p = pd.read_json(r)

r = json.dumps(r)
p = pd.read_json(r)
p.head(10)

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,SEX_LABEL,ETH_GROUP_LABEL,RACE_GROUP_LABEL,VET_GROUP_LABEL,EMPSZFI_LABEL,YEAR,...,PAYANN_F,FIRMPDEMP_S,FIRMPDEMP_S_F,RCPPDEMP_S,RCPPDEMP_S_F,EMP_S,EMP_S_F,PAYANN_S,PAYANN_S_F,us
1,0100000US,United States,00,Total for all sectors,Total,Total,Total,Total,All firms,2018,...,None,0.0,None,1.2,None,0.5,None,0.5,None,1
2,0100000US,United States,11,"Agriculture, forestry, fishing and hunting",Total,Total,Total,Total,All firms,2018,...,None,6.7,None,0.0,S,23.9,None,12.8,None,1
3,0100000US,United States,21,"Mining, quarrying, and oil and gas extraction",Total,Total,Total,Total,All firms,2018,...,None,0.8,None,0.0,S,5.4,None,6.0,None,1
4,0100000US,United States,23,Construction,Total,Total,Total,Total,All firms,2018,...,None,0.9,None,0.0,S,2.6,None,2.8,None,1
5,0100000US,United States,31-33,Manufacturing,Total,Total,Total,Total,All firms,2018,...,None,0.2,None,0.0,S,1.2,None,1.1,None,1
6,0100000US,United States,42,Wholesale trade,Total,Total,Total,Total,All firms,2018,...,None,0.3,None,0.0,S,1.7,None,1.7,None,1
7,0100000US,United States,44-45,Retail trade,Total,Total,Total,Total,All firms,2018,...,None,1.3,None,0.0,S,2.0,None,3.1,None,1
8,0100000US,United States,48-49,Transportation and warehousing,Total,Total,Total,Total,All firms,2018,...,None,0.4,None,0.0,S,2.6,None,2.4,None,1
9,0100000US,United States,51,Information,Total,Total,Total,Total,All firms,2018,...,None,0.9,None,0.0,S,1.8,None,2.0,None,1


,0,1,2,3
0,GEO_ID,NAME,RACE_GROUP,state
1,0400000US28,Mississippi,92,28
2,0400000US28,Mississippi,50,28
3,0400000US28,Mississippi,60,28
4,0400000US42,Pennsylvania,50,42
5,0400000US28,Mississippi,30,28
6,0400000US28,Mississippi,70,28
7,0400000US28,Mississippi,90,28
8,0400000US28,Mississippi,00,28
9,0400000US28,Mississippi,40,28
